In [ ]:
import torch
from transformers import BertTokenizer, BertModel, BertForMaskedLM




tokenizer = BertTokenizer.from_pretrained('DeepPavlov/rubert-base-cased')
model = BertForMaskedLM.from_pretrained('DeepPavlov/rubert-base-cased')
model.eval()
model.to('cuda')  


def predict_masked_sent(text, top_k=5):
    # Tokenize input
    text = "[CLS] %s [SEP]"%text
    tokenized_text = tokenizer.tokenize(text)
    masked_index = tokenized_text.index("[MASK]")
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    tokens_tensor = torch.tensor([indexed_tokens])
    tokens_tensor = tokens_tensor.to('cuda')    # if you have gpu

    # Predict all tokens
    with torch.no_grad():
        outputs = model(tokens_tensor)
        predictions = outputs[0]

    probs = torch.nn.functional.softmax(predictions[0, masked_index], dim=-1)
    top_k_weights, top_k_indices = torch.topk(probs, top_k, sorted=True)

    for i, pred_idx in enumerate(top_k_indices):
        predicted_token = tokenizer.convert_ids_to_tokens([pred_idx])[0]
        token_weight = top_k_weights[i]
        print("[MASK]: '%s'"%predicted_token, " | weights:", float(token_weight))

        
predict_masked_sent("My [MASK] is so cute.", top_k=5)


Limits exceeded: Not enough units in project to continue executions. Please contact your project admin.


_________________________________

In [4]:
predict_masked_sent("В дополнение утвержденного Главнокомандующим 18 мая 1919 года постановления Особого [MASK] о предоставлении учреждениям гражданского ведомства права перевозок по железным дорогам чинов, командируемых по службе, и грузов для надобностей Вооруженных Сил на Юге России по воинским предложениям предоставить указанным учреждениям право перевозок по воинским предложениям тех чинов и грузов по водным путям с соблюдением правил, установленных означенным постановлением для перевозок по железным дорогам, с тем чтобы перевозки водой за счет казны по воинским предложениям производились лишь в тех случаях, когда эти перевозки будут обходиться дешевле, чем перевозки между теми же пунктами по железным дорогам, или когда между соответствующими пунктами вовсе нет железных дорог", top_k=5)

[MASK]: 'совещания'  | weights: 0.723680853843689
[MASK]: 'отдела'  | weights: 0.0478539876639843
[MASK]: 'Комитета'  | weights: 0.029952943325042725
[MASK]: 'комитета'  | weights: 0.02658521756529808
[MASK]: 'Особ'  | weights: 0.01699681393802166


In [5]:
predict_masked_sent("В дополнение утвержденного Главнокомандующим 18 мая [MASK] года постановления Особого совещания о предоставлении учреждениям гражданского ведомства права перевозок по железным дорогам чинов, командируемых по службе, и грузов для надобностей Вооруженных Сил на Юге России по воинским предложениям предоставить указанным учреждениям право перевозок по воинским предложениям тех чинов и грузов по водным путям с соблюдением правил, установленных означенным постановлением для перевозок по железным дорогам, с тем чтобы перевозки водой за счет казны по воинским предложениям производились лишь в тех случаях, когда эти перевозки будут обходиться дешевле, чем перевозки между теми же пунктами по железным дорогам, или когда между соответствующими пунктами вовсе нет железных дорог", top_k=5)

[MASK]: '1916'  | weights: 0.07815678417682648
[MASK]: '1917'  | weights: 0.07105229794979095
[MASK]: '1912'  | weights: 0.06009657680988312
[MASK]: '1915'  | weights: 0.05519359931349754
[MASK]: '1918'  | weights: 0.04400081932544708


In [6]:
text = "Ю.О. [MASK]. Небывалая хозяйственная разруха, катастрофическое положение продовольственного дела в атмосфере непрекращающейся гражданской войны порождает в отсталых слоях населения угрожающее погромное настроение. При наличности веками слагавшихся национальных предрассудков настроение это под влиянием явно черносотенных, а подчас и примазавшихся к советской власти темных элементов выливается по преимуществу в форму антиеврейских погромов. Все враги революции, все сторонники прежнего режима прибегают к старому, испытанному оружию  антисемитской травле, науськиванию масс на евреев. А ныне стоящая у власти большевистская партия, своей демагогической агитацией развращавшая и развращающая массы, а всей своей правительственной политикой порождающая условия, систематически питающие погромные настроения, абсолютно не в состоянии, каковы бы ни были субъективные настроения ее руководителей, вести хоть сколько-нибудь успешную борьбу с погромной опасностью"

predict_masked_sent(text, top_k=5)

[MASK]: ')'  | weights: 0.02555733546614647
[MASK]: 'Смирнов'  | weights: 0.02148241177201271
[MASK]: 'Шмидт'  | weights: 0.02061973139643669
[MASK]: 'Смирнова'  | weights: 0.008209827356040478
[MASK]: 'Медведев'  | weights: 0.0077885910868644714


In [7]:
text = "Ю.О. Мартов. Небывалая хозяйственная разруха, катастрофическое положение продовольственного дела в атмосфере непрекращающейся гражданской войны порождает в отсталых слоях населения угрожающее погромное настроение. При наличности веками слагавшихся национальных предрассудков настроение это под влиянием явно черносотенных, а подчас и примазавшихся к советской власти темных элементов выливается по преимуществу в форму [MASK] погромов. Все враги революции, все сторонники прежнего режима прибегают к старому, испытанному оружию  антисемитской травле, науськиванию масс на евреев. А ныне стоящая у власти большевистская партия, своей демагогической агитацией развращавшая и развращающая массы, а всей своей правительственной политикой порождающая условия, систематически питающие погромные настроения, абсолютно не в состоянии, каковы бы ни были субъективные настроения ее руководителей, вести хоть сколько-нибудь успешную борьбу с погромной опасностью"

predict_masked_sent(text, top_k=5)

[MASK]: 'массовых'  | weights: 0.17981332540512085
[MASK]: 'городских'  | weights: 0.12120777368545532
[MASK]: 'революционных'  | weights: 0.07590944319963455
[MASK]: 'национальных'  | weights: 0.025601547211408615
[MASK]: 'политических'  | weights: 0.01969822868704796


In [8]:
text = "Ю.О. Мартов. Небывалая хозяйственная разруха, катастрофическое положение продовольственного дела в атмосфере непрекращающейся гражданской войны порождает в отсталых слоях населения угрожающее погромное настроение. При наличности веками слагавшихся национальных предрассудков настроение это под влиянием явно черносотенных, а подчас и примазавшихся к советской власти темных элементов выливается по преимуществу в форму антиеврейских погромов. Все враги революции, все сторонники прежнего режима прибегают к старому, испытанному оружию  антисемитской травле, науськиванию масс на евреев. А ныне стоящая у власти [MASK] партия, своей демагогической агитацией развращавшая и развращающая массы, а всей своей правительственной политикой порождающая условия, систематически питающие погромные настроения, абсолютно не в состоянии, каковы бы ни были субъективные настроения ее руководителей, вести хоть сколько-нибудь успешную борьбу с погромной опасностью."

predict_masked_sent(text, top_k=5)

[MASK]: 'коммунистическая'  | weights: 0.3035661578178406
[MASK]: 'политическая'  | weights: 0.07387485355138779
[MASK]: 'Коммунистическая'  | weights: 0.05963360145688057
[MASK]: 'правящая'  | weights: 0.05143214389681816
[MASK]: 'революционная'  | weights: 0.040280137211084366


In [9]:
text = "Ю.О. Мартов. Небывалая хозяйственная разруха, катастрофическое положение продовольственного дела в атмосфере непрекращающейся гражданской войны порождает в отсталых слоях населения угрожающее погромное настроение. При наличности веками слагавшихся национальных предрассудков настроение это под влиянием явно черносотенных, а подчас и примазавшихся к советской власти темных элементов выливается по преимуществу в форму антиеврейских погромов. Все враги [MASK], все сторонники прежнего режима прибегают к старому, испытанному оружию  антисемитской травле, науськиванию масс на евреев. А ныне стоящая у власти большевистская партия, своей демагогической агитацией развращавшая и развращающая массы, а всей своей правительственной политикой порождающая условия, систематически питающие погромные настроения, абсолютно не в состоянии, каковы бы ни были субъективные настроения ее руководителей, вести хоть сколько-нибудь успешную борьбу с погромной опасностью."

predict_masked_sent(text, top_k=5)

[MASK]: 'народа'  | weights: 0.28623121976852417
[MASK]: 'прошлого'  | weights: 0.11077520996332169
[MASK]: 'революции'  | weights: 0.05247101932764053
[MASK]: 'СССР'  | weights: 0.04751133173704147
[MASK]: 'России'  | weights: 0.030653513967990875


In [10]:
text = "Ю.О. Мартов. Небывалая хозяйственная разруха, катастрофическое положение продовольственного дела в атмосфере непрекращающейся гражданской войны порождает в отсталых слоях населения угрожающее погромное настроение. При наличности веками слагавшихся [MASK] предрассудков настроение это под влиянием явно черносотенных, а подчас и примазавшихся к советской власти темных элементов выливается по преимуществу в форму антиеврейских погромов. Все враги революции, все сторонники прежнего режима прибегают к старому, испытанному оружию  антисемитской травле, науськиванию масс на евреев. А ныне стоящая у власти большевистская партия, своей демагогической агитацией развращавшая и развращающая массы, а всей своей правительственной политикой порождающая условия, систематически питающие погромные настроения, абсолютно не в состоянии, каковы бы ни были субъективные настроения ее руководителей, вести хоть сколько-нибудь успешную борьбу с погромной опасностью."

predict_masked_sent(text, top_k=5)

[MASK]: 'национальных'  | weights: 0.16074953973293304
[MASK]: 'еврейских'  | weights: 0.11378365755081177
[MASK]: 'народных'  | weights: 0.07702159881591797
[MASK]: 'антисемитских'  | weights: 0.07677562534809113
[MASK]: 'религиозных'  | weights: 0.04293926805257797


In [11]:
text = "Ю.О. Мартов. Небывалая хозяйственная разруха, катастрофическое положение продовольственного дела в атмосфере непрекращающейся гражданской войны порождает в отсталых слоях населения угрожающее погромное настроение. При наличности веками слагавшихся национальных предрассудков настроение это под влиянием явно черносотенных, а подчас и примазавшихся к [MASK] власти темных элементов выливается по преимуществу в форму антиеврейских погромов. Все враги революции, все сторонники прежнего режима прибегают к старому, испытанному оружию  антисемитской травле, науськиванию масс на евреев. А ныне стоящая у власти большевистская партия, своей демагогической агитацией развращавшая и развращающая массы, а всей своей правительственной политикой порождающая условия, систематически питающие погромные настроения, абсолютно не в состоянии, каковы бы ни были субъективные настроения ее руководителей, вести хоть сколько-нибудь успешную борьбу с погромной опасностью."
predict_masked_sent(text, top_k=5)

[MASK]: 'советской'  | weights: 0.1941501945257187
[MASK]: 'новой'  | weights: 0.14474697411060333
[MASK]: 'царской'  | weights: 0.11698414385318756
[MASK]: 'Советской'  | weights: 0.08458054065704346
[MASK]: 'центральной'  | weights: 0.031603507697582245


___________________

In [12]:


text = "Передайте VIII съезду уполномоченных [MASK] обществ Мою сердечную благодарность за их молитвы, благопожелания и выраженные Государыням Императрицам, Мне и Наследнику Цесаревичу чувства; уверен, что верное заветам старины Русское дворянство и впредь всегда будет служить опорой Престола в деле мирного развития великой нашей России."

predict_masked_sent(text, top_k=5)

[MASK]: 'дворянских'  | weights: 0.5395142436027527
[MASK]: 'акционерных'  | weights: 0.10131962597370148
[MASK]: 'сельских'  | weights: 0.05199793726205826
[MASK]: 'русских'  | weights: 0.024523934349417686
[MASK]: 'вольных'  | weights: 0.024025574326515198


In [13]:
text = "Передайте VIII съезду уполномоченных 31 дворянских обществ Мою сердечную благодарность за их молитвы, благопожелания и выраженные Государыням Императрицам, Мне и Наследнику Цесаревичу чувства; уверен, что верное заветам старины Русское дворянство и впредь всегда будет служить опорой [MASK] в деле мирного развития великой нашей России."

predict_masked_sent(text, top_k=5)

[MASK]: 'государства'  | weights: 0.11667516082525253
[MASK]: 'народа'  | weights: 0.09200868755578995
[MASK]: 'России'  | weights: 0.06667143106460571
[MASK]: 'общества'  | weights: 0.04329853504896164
[MASK]: 'Церкви'  | weights: 0.027146251872181892


In [14]:
text = "Передайте VIII съезду уполномоченных 31 дворянских обществ Мою сердечную благодарность за их молитвы, благопожелания и выраженные Государыням Императрицам, [MASK] и Наследнику Цесаревичу чувства; уверен, что верное заветам старины Русское дворянство и впредь всегда будет служить опорой Престола в деле мирного развития великой нашей России."

predict_masked_sent(text, top_k=5)

[MASK]: 'Вам'  | weights: 0.3465823829174042
[MASK]: 'императрице'  | weights: 0.07070856541395187
[MASK]: 'Героям'  | weights: 0.03499092906713486
[MASK]: 'министрам'  | weights: 0.01821635290980339
[MASK]: 'наследникам'  | weights: 0.015892162919044495


--------------------------


In [15]:
text = "Петроградский [MASK] послал в эти дни к вам, московским рабочим, своих делегатов для того, чтобы связать свое движение с вашим, для того, чтобы рассказать вам о нуждах и планах петроградских пролетариев, и обратно привезти ваше сочувствие и вашу поддержку. Петроградский пролетариат оказался в чрезвычайно тяжелом положении. Испытавший на себе наиболее  остро все последствия дикой большевистской политики, потерявший в результате этой политики силу и независимость своих организаций, он, фактически  безоружный и беззащитный, столкнулся лицом к лицу с последними бедствиями, надвинувшимися на Петроград. Обреченный город начал эвакуироваться. Совет Народных Комиссаров покинул Петроград. Начались расчеты на заводах. Во всех вопросах — эвакуации, безработицы, продовольствия,  охраны безопасности населения — рабочий класс воочию увидел одну разруху, полную дезорганизацию. Перевыборы в Совете фактически были запрещены и оторвавшись совершенно от рабочей массы, эти большевистские канцелярии бездействовали, или сами усиливали дезорганизацию. Деятельность профессиональных союзов, заводских комитетов была извращена и разрушена, кооперативы — оттерты от широкой работы."

predict_masked_sent(text, top_k=5)

[MASK]: 'Совет'  | weights: 0.7407854795455933
[MASK]: 'совет'  | weights: 0.1600060760974884
[MASK]: 'ВРК'  | weights: 0.03879310190677643
[MASK]: 'комитет'  | weights: 0.01608281396329403
[MASK]: 'градоначальник'  | weights: 0.007747064344584942


In [32]:
text = "Петроградский пролетариат послал в эти дни к вам, московским [MASK], своих делегатов для того, чтобы связать свое движение с вашим, для того, чтобы рассказать вам о нуждах и планах петроградских пролетариев, и обратно привезти ваше сочувствие и вашу поддержку. Петроградский пролетариат оказался в чрезвычайно тяжелом положении. Испытавший на себе наиболее  остро все последствия дикой большевистской политики, потерявший в результате этой политики силу и независимость своих организаций, он, фактически  безоружный и беззащитный, столкнулся лицом к лицу с последними бедствиями, надвинувшимися на Петроград. Обреченный город начал эвакуироваться. Совет Народных Комиссаров покинул Петроград. Начались расчеты на заводах. Во всех вопросах — эвакуации, безработицы, продовольствия,  охраны безопасности населения — рабочий класс воочию увидел одну разруху, полную дезорганизацию. Перевыборы в Совете фактически были запрещены и оторвавшись совершенно от рабочей массы, эти большевистские канцелярии бездействовали, или сами усиливали дезорганизацию. Деятельность профессиональных союзов, заводских комитетов была извращена и разрушена, кооперативы — оттерты от широкой работы."

predict_masked_sent(text, top_k=5)

[MASK]: 'рабочим'  | weights: 0.20496109127998352
[MASK]: 'властям'  | weights: 0.102394238114357
[MASK]: 'большевикам'  | weights: 0.09463527053594589
[MASK]: 'людям'  | weights: 0.06855359673500061
[MASK]: 'городам'  | weights: 0.022449610754847527


In [33]:
text = "Петроградский пролетариат послал в эти дни к вам, московским рабочим, своих делегатов для того, чтобы связать свое движение с вашим, для того, чтобы рассказать вам о нуждах и планах петроградских пролетариев, и обратно привезти ваше сочувствие и вашу поддержку. Петроградский пролетариат оказался в чрезвычайно тяжелом положении. Испытавший на себе наиболее  остро все последствия дикой [MASK] политики, потерявший в результате этой политики силу и независимость своих организаций, он, фактически  безоружный и беззащитный, столкнулся лицом к лицу с последними бедствиями, надвинувшимися на Петроград. Обреченный город начал эвакуироваться. Совет Народных Комиссаров покинул Петроград. Начались расчеты на заводах. Во всех вопросах — эвакуации, безработицы, продовольствия,  охраны безопасности населения — рабочий класс воочию увидел одну разруху, полную дезорганизацию. Перевыборы в Совете фактически были запрещены и оторвавшись совершенно от рабочей массы, эти большевистские канцелярии бездействовали, или сами усиливали дезорганизацию. Деятельность профессиональных союзов, заводских комитетов была извращена и разрушена, кооперативы — оттерты от широкой работы."

predict_masked_sent(text, top_k=5)

[MASK]: 'большевистской'  | weights: 0.2727024555206299
[MASK]: 'революционной'  | weights: 0.07594580203294754
[MASK]: 'внутренней'  | weights: 0.04247301444411278
[MASK]: 'внешней'  | weights: 0.03928108140826225
[MASK]: 'буржуазной'  | weights: 0.03043038211762905


In [34]:
text = "Петроградский пролетариат послал в эти дни к вам, московским рабочим, своих делегатов для того, чтобы связать свое движение с вашим, для того, чтобы рассказать вам о нуждах и планах петроградских пролетариев, и обратно привезти ваше сочувствие и вашу поддержку. Петроградский пролетариат оказался в чрезвычайно тяжелом положении. Испытавший на себе наиболее  остро все последствия дикой большевистской политики, потерявший в результате этой политики силу и независимость своих организаций, он, фактически  безоружный и беззащитный, столкнулся лицом к лицу с последними бедствиями, надвинувшимися на [MASK]. Обреченный город начал эвакуироваться. Совет Народных Комиссаров покинул Петроград. Начались расчеты на заводах. Во всех вопросах — эвакуации, безработицы, продовольствия,  охраны безопасности населения — рабочий класс воочию увидел одну разруху, полную дезорганизацию. Перевыборы в Совете фактически были запрещены и оторвавшись совершенно от рабочей массы, эти большевистские канцелярии бездействовали, или сами усиливали дезорганизацию. Деятельность профессиональных союзов, заводских комитетов была извращена и разрушена, кооперативы — оттерты от широкой работы."

predict_masked_sent(text, top_k=5)

[MASK]: 'Петроград'  | weights: 0.3993767499923706
[MASK]: 'город'  | weights: 0.06557609140872955
[MASK]: 'рабочих'  | weights: 0.06510810554027557
[MASK]: 'Россию'  | weights: 0.06280786544084549
[MASK]: 'большевиков'  | weights: 0.034294743090867996


In [35]:
text = "Петроградский пролетариат послал в эти дни к вам, московским рабочим, своих делегатов для того, чтобы связать свое движение с вашим, для того, чтобы рассказать вам о нуждах и планах петроградских пролетариев, и обратно привезти ваше сочувствие и вашу поддержку. Петроградский пролетариат оказался в чрезвычайно тяжелом положении. Испытавший на себе наиболее  остро все последствия дикой большевистской политики, потерявший в результате этой политики силу и независимость своих организаций, он, фактически  безоружный и беззащитный, столкнулся лицом к лицу с последними бедствиями, надвинувшимися на Петроград. Обреченный город начал эвакуироваться. Совет Народных [MASK] покинул Петроград. Начались расчеты на заводах. Во всех вопросах — эвакуации, безработицы, продовольствия,  охраны безопасности населения — рабочий класс воочию увидел одну разруху, полную дезорганизацию. Перевыборы в Совете фактически были запрещены и оторвавшись совершенно от рабочей массы, эти большевистские канцелярии бездействовали, или сами усиливали дезорганизацию. Деятельность профессиональных союзов, заводских комитетов была извращена и разрушена, кооперативы — оттерты от широкой работы."

predict_masked_sent(text, top_k=5)

[MASK]: 'Комиссаров'  | weights: 0.9703355431556702
[MASK]: 'комиссаров'  | weights: 0.017781199887394905
[MASK]: 'рабочих'  | weights: 0.0031793704256415367
[MASK]: 'депутатов'  | weights: 0.003156783292070031
[MASK]: 'представителей'  | weights: 0.000998733681626618


In [36]:
text = "Петроградский пролетариат послал в эти дни к вам, московским рабочим, своих делегатов для того, чтобы связать свое движение с вашим, для того, чтобы рассказать вам о нуждах и планах петроградских пролетариев, и обратно привезти ваше сочувствие и вашу поддержку. Петроградский пролетариат оказался в чрезвычайно тяжелом положении. Испытавший на себе наиболее  остро все последствия дикой большевистской политики, потерявший в результате этой политики силу и независимость своих организаций, он, фактически  безоружный и беззащитный, столкнулся лицом к лицу с последними бедствиями, надвинувшимися на Петроград. Обреченный город начал эвакуироваться. Совет Народных Комиссаров покинул Петроград. Начались расчеты на заводах. Во всех вопросах — эвакуации, [MASK], продовольствия,  охраны безопасности населения — рабочий класс воочию увидел одну разруху, полную дезорганизацию. Перевыборы в Совете фактически были запрещены и оторвавшись совершенно от рабочей массы, эти большевистские канцелярии бездействовали, или сами усиливали дезорганизацию. Деятельность профессиональных союзов, заводских комитетов была извращена и разрушена, кооперативы — оттерты от широкой работы."

predict_masked_sent(text, top_k=5)

[MASK]: 'транспорта'  | weights: 0.14210428297519684
[MASK]: 'торговли'  | weights: 0.051051344722509384
[MASK]: 'снабжения'  | weights: 0.04412592574954033
[MASK]: 'труда'  | weights: 0.04384370520710945
[MASK]: 'отопления'  | weights: 0.04298880323767662


In [37]:
text = "Петроградский пролетариат послал в эти дни к вам, московским рабочим, своих делегатов для того, чтобы связать свое движение с вашим, для того, чтобы рассказать вам о нуждах и планах петроградских пролетариев, и обратно привезти ваше сочувствие и вашу поддержку. Петроградский пролетариат оказался в чрезвычайно тяжелом положении. Испытавший на себе наиболее  остро все последствия дикой большевистской политики, потерявший в результате этой политики силу и независимость своих организаций, он, фактически  безоружный и беззащитный, столкнулся лицом к лицу с последними бедствиями, надвинувшимися на Петроград. Обреченный город начал эвакуироваться. Совет Народных Комиссаров покинул Петроград. Начались расчеты на заводах. Во всех вопросах — эвакуации, безработицы, [MASK],  охраны безопасности населения — рабочий класс воочию увидел одну разруху, полную дезорганизацию. Перевыборы в Совете фактически были запрещены и оторвавшись совершенно от рабочей массы, эти большевистские канцелярии бездействовали, или сами усиливали дезорганизацию. Деятельность профессиональных союзов, заводских комитетов была извращена и разрушена, кооперативы — оттерты от широкой работы."

predict_masked_sent(text, top_k=5)

[MASK]: 'труда'  | weights: 0.0727122351527214
[MASK]: 'мобилизации'  | weights: 0.05831175297498703
[MASK]: 'забастовки'  | weights: 0.03486841917037964
[MASK]: 'питания'  | weights: 0.032730307430028915
[MASK]: 'торговли'  | weights: 0.02662554197013378


In [38]:
text = "Петроградский пролетариат послал в эти дни к вам, московским рабочим, своих делегатов для того, чтобы связать свое движение с вашим, для того, чтобы рассказать вам о нуждах и планах петроградских пролетариев, и обратно привезти ваше сочувствие и вашу поддержку. Петроградский пролетариат оказался в чрезвычайно тяжелом положении. Испытавший на себе наиболее  остро все последствия дикой большевистской политики, потерявший в результате этой политики силу и независимость своих организаций, он, фактически  безоружный и беззащитный, столкнулся лицом к лицу с последними бедствиями, надвинувшимися на Петроград. Обреченный город начал эвакуироваться. Совет Народных Комиссаров покинул Петроград. Начались расчеты на заводах. Во всех вопросах — эвакуации, безработицы, продовольствия,  охраны безопасности населения — рабочий [MASK] воочию увидел одну разруху, полную дезорганизацию. Перевыборы в Совете фактически были запрещены и оторвавшись совершенно от рабочей массы, эти большевистские канцелярии бездействовали, или сами усиливали дезорганизацию. Деятельность профессиональных союзов, заводских комитетов была извращена и разрушена, кооперативы — оттерты от широкой работы."

predict_masked_sent(text, top_k=5)

[MASK]: 'класс'  | weights: 0.44250309467315674
[MASK]: 'Совет'  | weights: 0.07160494476556778
[MASK]: 'народ'  | weights: 0.07027880102396011
[MASK]: 'Петроград'  | weights: 0.04718133434653282
[MASK]: 'совет'  | weights: 0.024265386164188385


In [39]:
text = "Петроградский пролетариат послал в эти дни к вам, московским рабочим, своих делегатов для того, чтобы связать свое движение с вашим, для того, чтобы рассказать вам о нуждах и планах петроградских пролетариев, и обратно привезти ваше сочувствие и вашу поддержку. Петроградский пролетариат оказался в чрезвычайно тяжелом положении. Испытавший на себе наиболее  остро все последствия дикой большевистской политики, потерявший в результате этой политики силу и независимость своих организаций, он, фактически  безоружный и беззащитный, столкнулся лицом к лицу с последними бедствиями, надвинувшимися на Петроград. Обреченный город начал эвакуироваться. Совет Народных Комиссаров покинул Петроград. Начались расчеты на заводах. Во всех вопросах — эвакуации, безработицы, продовольствия,  охраны безопасности населения — рабочий класс воочию увидел одну разруху, полную дезорганизацию. Перевыборы в Совете фактически были запрещены и оторвавшись совершенно от рабочей массы, эти большевистские канцелярии бездействовали, или сами усиливали дезорганизацию. Деятельность профессиональных союзов, заводских [MASK] была извращена и разрушена, кооперативы — оттерты от широкой работы."

predict_masked_sent(text, top_k=5)

[MASK]: 'рабочих'  | weights: 0.25603994727134705
[MASK]: 'профсоюзов'  | weights: 0.09124258160591125
[MASK]: 'организаций'  | weights: 0.07639528065919876
[MASK]: 'комитетов'  | weights: 0.07192472368478775
[MASK]: 'союзов'  | weights: 0.0644291415810585


In [40]:
text = "Петроградский пролетариат послал в эти дни к вам, московским рабочим, своих делегатов для того, чтобы связать свое движение с вашим, для того, чтобы рассказать вам о нуждах и планах петроградских пролетариев, и обратно привезти ваше сочувствие и вашу поддержку. Петроградский пролетариат оказался в чрезвычайно тяжелом положении. Испытавший на себе наиболее  остро все последствия дикой большевистской политики, потерявший в результате этой политики силу и независимость своих организаций, он, фактически  безоружный и беззащитный, столкнулся лицом к лицу с последними бедствиями, надвинувшимися на Петроград. Обреченный город начал эвакуироваться. Совет Народных Комиссаров покинул Петроград. Начались расчеты на заводах. Во всех вопросах — эвакуации, безработицы, продовольствия,  охраны безопасности населения — рабочий класс воочию увидел одну разруху, полную дезорганизацию. Перевыборы в [MASK] фактически были запрещены и оторвавшись совершенно от рабочей массы, эти большевистские канцелярии бездействовали, или сами усиливали дезорганизацию. Деятельность профессиональных союзов, заводских комитетов была извращена и разрушена, кооперативы — оттерты от широкой работы."

predict_masked_sent(text, top_k=5)

[MASK]: 'городах'  | weights: 0.24273759126663208
[MASK]: 'Петрограде'  | weights: 0.11670693010091782
[MASK]: 'губернии'  | weights: 0.07192368060350418
[MASK]: 'Советы'  | weights: 0.06882932037115097
[MASK]: 'городе'  | weights: 0.04892968013882637


_______________________

In [43]:
text = "Докладчиком по вопросу о [MASK] с контрреволюцией выступает г. Зиновьев. - Он предлагает рабочей секции принять резолюцию, проект которой опубликован в газете «Правда» от 24 июня. Нам говорят о контрреволюции в «Маленькой газете» и в «Новом времени». Но это не настоящая контрреволюция.  Это правильнее назвать [контрреволюционным хулиганством. Настоящая  контрреволюция гнездится в том классе, интересам которого вредит революция,  и этот класс есть буржуазия. Не случайность, что представителем буржуазии  является П.Н. Милюков. Сегодня буржуазия засвидетельствовала нам свою контрреволюционность. Из состава коалиционного министерства ушли министры-кадеты. Уход кадетов из министерства — это не доктринерство, как  хотят изобразить некоторые, а глубокое понимание своих классовых интересов.  Капиталисты ведут страну к гибели. Об этом пишут в органе министров-социалистов,  в «Рабочей газете»...3 На точке зрения партии кадетов. Из-за чего, собственно, произошел последний министерский кризис? Кадетский министр, министр просвещения Мануйлов, отказался сместить чиновников контрреволюционеров в своем ведомстве. Впрочем, не сместили чиновников и министры-социалисты Керенский и Переверзев по министерству юстиции. Отказ Мануйлова подчистить свое министерство привел к тому, что кадеты ухватились, как за повод, за Украинскую раду и вышли из состава коалиционного министерства. А буржуазная печать разве не отравляет деревню?"

predict_masked_sent(text, top_k=5)

[MASK]: 'борьбе'  | weights: 0.9256462454795837
[MASK]: 'связи'  | weights: 0.028877809643745422
[MASK]: 'ситуации'  | weights: 0.022666171193122864
[MASK]: 'войне'  | weights: 0.0038073277100920677
[MASK]: 'связях'  | weights: 0.001592713175341487


In [45]:
text = "Докладчиком по вопросу о борьбе с контрреволюцией выступает [MASK]. - Он предлагает рабочей секции принять резолюцию, проект которой опубликован в газете «Правда» от 24 июня. Нам говорят о контрреволюции в «Маленькой газете» и в «Новом времени». Но это не настоящая контрреволюция.  Это правильнее назвать [контрреволюционным хулиганством. Настоящая  контрреволюция гнездится в том классе, интересам которого вредит революция,  и этот класс есть буржуазия. Не случайность, что представителем буржуазии  является П.Н. Милюков. Сегодня буржуазия засвидетельствовала нам свою контрреволюционность. Из состава коалиционного министерства ушли министры-кадеты. Уход кадетов из министерства — это не доктринерство, как  хотят изобразить некоторые, а глубокое понимание своих классовых интересов.  Капиталисты ведут страну к гибели. Об этом пишут в органе министров-социалистов,  в «Рабочей газете»...3 На точке зрения партии кадетов. Из-за чего, собственно, произошел последний министерский кризис? Кадетский министр, министр просвещения Мануйлов, отказался сместить чиновников контрреволюционеров в своем ведомстве. Впрочем, не сместили чиновников и министры-социалисты Керенский и Переверзев по министерству юстиции. Отказ Мануйлова подчистить свое министерство привел к тому, что кадеты ухватились, как за повод, за Украинскую раду и вышли из состава коалиционного министерства. А буржуазная печать разве не отравляет деревню?"

predict_masked_sent(text, top_k=5)

[MASK]: 'Троцкий'  | weights: 0.21871379017829895
[MASK]: 'Ленин'  | weights: 0.045647621154785156
[MASK]: 'он'  | weights: 0.04305226728320122
[MASK]: 'Молотов'  | weights: 0.0302524846047163
[MASK]: 'Каменев'  | weights: 0.020856384187936783


In [46]:
text = "Докладчиком по вопросу о борьбе с контрреволюцией выступает г. Зиновьев. - Он предлагает рабочей секции принять [MASK], проект которой опубликован в газете «Правда» от 24 июня. Нам говорят о контрреволюции в «Маленькой газете» и в «Новом времени». Но это не настоящая контрреволюция.  Это правильнее назвать [контрреволюционным хулиганством. Настоящая  контрреволюция гнездится в том классе, интересам которого вредит революция,  и этот класс есть буржуазия. Не случайность, что представителем буржуазии  является П.Н. Милюков. Сегодня буржуазия засвидетельствовала нам свою контрреволюционность. Из состава коалиционного министерства ушли министры-кадеты. Уход кадетов из министерства — это не доктринерство, как  хотят изобразить некоторые, а глубокое понимание своих классовых интересов.  Капиталисты ведут страну к гибели. Об этом пишут в органе министров-социалистов,  в «Рабочей газете»...3 На точке зрения партии кадетов. Из-за чего, собственно, произошел последний министерский кризис? Кадетский министр, министр просвещения Мануйлов, отказался сместить чиновников контрреволюционеров в своем ведомстве. Впрочем, не сместили чиновников и министры-социалисты Керенский и Переверзев по министерству юстиции. Отказ Мануйлова подчистить свое министерство привел к тому, что кадеты ухватились, как за повод, за Украинскую раду и вышли из состава коалиционного министерства. А буржуазная печать разве не отравляет деревню?"

predict_masked_sent(text, top_k=5)

[MASK]: 'резолюцию'  | weights: 0.8393383026123047
[MASK]: 'программу'  | weights: 0.0525420606136322
[MASK]: 'статью'  | weights: 0.021305616945028305
[MASK]: 'декларацию'  | weights: 0.020115504041314125
[MASK]: 'конституцию'  | weights: 0.007121585309505463


In [47]:
text = "Докладчиком по вопросу о борьбе с контрреволюцией выступает г. Зиновьев. - Он предлагает рабочей секции принять резолюцию, проект которой опубликован в газете [MASK] от 24 июня. Нам говорят о контрреволюции в «Маленькой газете» и в «Новом времени». Но это не настоящая контрреволюция.  Это правильнее назвать [контрреволюционным хулиганством. Настоящая  контрреволюция гнездится в том классе, интересам которого вредит революция,  и этот класс есть буржуазия. Не случайность, что представителем буржуазии  является П.Н. Милюков. Сегодня буржуазия засвидетельствовала нам свою контрреволюционность. Из состава коалиционного министерства ушли министры-кадеты. Уход кадетов из министерства — это не доктринерство, как  хотят изобразить некоторые, а глубокое понимание своих классовых интересов.  Капиталисты ведут страну к гибели. Об этом пишут в органе министров-социалистов,  в «Рабочей газете»...3 На точке зрения партии кадетов. Из-за чего, собственно, произошел последний министерский кризис? Кадетский министр, министр просвещения Мануйлов, отказался сместить чиновников контрреволюционеров в своем ведомстве. Впрочем, не сместили чиновников и министры-социалисты Керенский и Переверзев по министерству юстиции. Отказ Мануйлова подчистить свое министерство привел к тому, что кадеты ухватились, как за повод, за Украинскую раду и вышли из состава коалиционного министерства. А буржуазная печать разве не отравляет деревню?"

predict_masked_sent(text, top_k=5)

[MASK]: 'Правда'  | weights: 0.16135641932487488
[MASK]: 'Правды'  | weights: 0.12707901000976562
[MASK]: 'МК'  | weights: 0.08525214344263077
[MASK]: 'Ленина'  | weights: 0.046524833887815475
[MASK]: 'Известия'  | weights: 0.03996114432811737


In [48]:
text = "Докладчиком по вопросу о борьбе с контрреволюцией выступает г. Зиновьев. - Он предлагает рабочей секции принять резолюцию, проект которой опубликован в газете «Правда» от 24 июня [MASK] года. Нам говорят о контрреволюции в «Маленькой газете» и в «Новом времени». Но это не настоящая контрреволюция.  Это правильнее назвать [контрреволюционным хулиганством. Настоящая  контрреволюция гнездится в том классе, интересам которого вредит революция,  и этот класс есть буржуазия. Не случайность, что представителем буржуазии  является П.Н. Милюков. Сегодня буржуазия засвидетельствовала нам свою контрреволюционность. Из состава коалиционного министерства ушли министры-кадеты. Уход кадетов из министерства — это не доктринерство, как  хотят изобразить некоторые, а глубокое понимание своих классовых интересов.  Капиталисты ведут страну к гибели. Об этом пишут в органе министров-социалистов,  в «Рабочей газете»...3 На точке зрения партии кадетов. Из-за чего, собственно, произошел последний министерский кризис? Кадетский министр, министр просвещения Мануйлов, отказался сместить чиновников контрреволюционеров в своем ведомстве. Впрочем, не сместили чиновников и министры-социалисты Керенский и Переверзев по министерству юстиции. Отказ Мануйлова подчистить свое министерство привел к тому, что кадеты ухватились, как за повод, за Украинскую раду и вышли из состава коалиционного министерства. А буржуазная печать разве не отравляет деревню?"

predict_masked_sent(text, top_k=5)

[MASK]: '1917'  | weights: 0.5734045505523682
[MASK]: '1918'  | weights: 0.11345211416482925
[MASK]: '1916'  | weights: 0.054205529391765594
[MASK]: '1905'  | weights: 0.035978738218545914
[MASK]: '1919'  | weights: 0.033047884702682495


In [49]:
text = "Докладчиком по вопросу о борьбе с контрреволюцией выступает г. Зиновьев. - Он предлагает рабочей секции принять резолюцию, проект которой опубликован в газете «Правда» от 24 июня. Нам говорят о контрреволюции в «Маленькой газете» и в «Новом времени». Но это не настоящая [MASK].  Это правильнее назвать [контрреволюционным хулиганством. Настоящая  контрреволюция гнездится в том [MASK], интересам которого вредит революция,  и этот класс есть буржуазия. Не случайность, что представителем буржуазии  является П.Н. Милюков. Сегодня буржуазия засвидетельствовала нам свою контрреволюционность. Из состава коалиционного министерства ушли министры-кадеты. Уход кадетов из министерства — это не доктринерство, как  хотят изобразить некоторые, а глубокое понимание своих классовых интересов.  Капиталисты ведут страну к гибели. Об этом пишут в органе министров-социалистов,  в «Рабочей газете»...3 На точке зрения партии кадетов. Из-за чего, собственно, произошел последний министерский кризис? Кадетский министр, министр просвещения Мануйлов, отказался сместить чиновников контрреволюционеров в своем ведомстве. Впрочем, не сместили чиновников и министры-социалисты Керенский и Переверзев по министерству юстиции. Отказ Мануйлова подчистить свое министерство привел к тому, что кадеты ухватились, как за повод, за Украинскую раду и вышли из состава коалиционного министерства. А буржуазная печать разве не отравляет деревню?"

predict_masked_sent(text, top_k=5)

[MASK]: 'революция'  | weights: 0.39213430881500244
[MASK]: 'пропаганда'  | weights: 0.09099549055099487
[MASK]: 'ложь'  | weights: 0.06272251904010773
[MASK]: 'провокация'  | weights: 0.03570496663451195
[MASK]: 'война'  | weights: 0.028383122757077217


In [50]:
text = "Докладчиком по вопросу о борьбе с контрреволюцией выступает г. Зиновьев. - Он предлагает рабочей секции принять резолюцию, проект которой опубликован в газете «Правда» от 24 июня. Нам говорят о контрреволюции в «Маленькой газете» и в «Новом времени». Но это не настоящая контрреволюция.  Это правильнее назвать [контрреволюционным хулиганством. Настоящая  контрреволюция гнездится в том классе, интересам которого вредит [MASK],  и этот класс есть буржуазия. Не случайность, что представителем буржуазии  является П.Н. Милюков. Сегодня буржуазия засвидетельствовала нам свою контрреволюционность. Из состава коалиционного министерства ушли министры-кадеты. Уход кадетов из министерства — это не доктринерство, как  хотят изобразить некоторые, а глубокое понимание своих классовых интересов.  Капиталисты ведут страну к гибели. Об этом пишут в органе министров-социалистов,  в «Рабочей газете»...3 На точке зрения партии кадетов. Из-за чего, собственно, произошел последний министерский кризис? Кадетский министр, министр просвещения Мануйлов, отказался сместить чиновников контрреволюционеров в своем ведомстве. Впрочем, не сместили чиновников и министры-социалисты Керенский и Переверзев по министерству юстиции. Отказ Мануйлова подчистить свое министерство привел к тому, что кадеты ухватились, как за повод, за Украинскую раду и вышли из состава коалиционного министерства. А буржуазная печать разве не отравляет деревню?"

predict_masked_sent(text, top_k=5)

[MASK]: 'революция'  | weights: 0.31102994084358215
[MASK]: 'государство'  | weights: 0.05612494423985481
[MASK]: 'правительство'  | weights: 0.05588949844241142
[MASK]: 'власть'  | weights: 0.0339001826941967
[MASK]: 'народ'  | weights: 0.03143901377916336


In [ ]:
text = "Докладчиком по вопросу о борьбе с контрреволюцией выступает г. Зиновьев. - Он предлагает рабочей секции принять резолюцию, проект которой опубликован в газете «Правда» от 24 июня. Нам говорят о контрреволюции в «Маленькой газете» и в «Новом времени». Но это не настоящая контрреволюция.  Это правильнее назвать [контрреволюционным хулиганством. Настоящая  контрреволюция гнездится в том классе, интересам которого вредит революция,  и этот класс есть буржуазия. Не случайность, что представителем буржуазии  является П.Н. Милюков. Сегодня буржуазия засвидетельствовала нам свою [MASK]. Из состава коалиционного министерства ушли министры-кадеты. Уход кадетов из министерства — это не доктринерство, как  хотят изобразить некоторые, а глубокое понимание своих классовых интересов.  Капиталисты ведут страну к гибели. Об этом пишут в органе министров-социалистов,  в «Рабочей газете»...3 На точке зрения партии кадетов. Из-за чего, собственно, произошел последний министерский кризис? Кадетский министр, министр просвещения Мануйлов, отказался сместить чиновников контрреволюционеров в своем ведомстве. Впрочем, не сместили чиновников и министры-социалисты Керенский и Переверзев по министерству юстиции. Отказ Мануйлова подчистить свое министерство привел к тому, что кадеты ухватились, как за повод, за Украинскую раду и вышли из состава коалиционного министерства. А буржуазная печать разве не отравляет деревню?"

predict_masked_sent(text, top_k=5)

[MASK]: 'победу'  | weights: 0.08724050968885422
[MASK]: 'революцию'  | weights: 0.07854128628969193
[MASK]: 'власть'  | weights: 0.06741076707839966
[MASK]: 'волю'  | weights: 0.0581209734082222
[MASK]: 'ответственность'  | weights: 0.04130861535668373


In [54]:
text = "Докладчиком по вопросу о борьбе с контрреволюцией выступает г. Зиновьев. - Он предлагает рабочей секции принять резолюцию, проект которой опубликован в газете «Правда» от 24 июня. Нам говорят о контрреволюции в «Маленькой газете» и в «Новом времени». Но это не настоящая контрреволюция.  Это правильнее назвать [контрреволюционным хулиганством. Настоящая  контрреволюция гнездится в том классе, интересам которого вредит революция,  и этот класс есть буржуазия. Не случайность, что представителем буржуазии  является П.Н. Милюков. Сегодня буржуазия засвидетельствовала нам свою контрреволюционность. Из состава коалиционного министерства ушли министры-кадеты. Уход кадетов из [MASK] — это не доктринерство, как  хотят изобразить некоторые, а глубокое понимание своих классовых интересов.  Капиталисты ведут страну к гибели. Об этом пишут в органе министров-социалистов,  в «Рабочей газете»...3 На точке зрения партии кадетов. Из-за чего, собственно, произошел последний министерский кризис? Кадетский министр, министр просвещения Мануйлов, отказался сместить чиновников контрреволюционеров в своем ведомстве. Впрочем, не сместили чиновников и министры-социалисты Керенский и Переверзев по министерству юстиции. Отказ Мануйлова подчистить свое министерство привел к тому, что кадеты ухватились, как за повод, за Украинскую раду и вышли из состава коалиционного министерства. А буржуазная печать разве не отравляет деревню?"
predict_masked_sent(text, top_k=5)

[MASK]: 'министерства'  | weights: 0.4570891559123993
[MASK]: 'правительства'  | weights: 0.2554358243942261
[MASK]: 'кабинета'  | weights: 0.07735803723335266
[MASK]: 'партии'  | weights: 0.029926806688308716
[MASK]: 'него'  | weights: 0.026636194437742233


In [55]:
text = "Докладчиком по вопросу о борьбе с контрреволюцией выступает г. Зиновьев. - Он предлагает рабочей секции принять резолюцию, проект которой опубликован в газете «Правда» от 24 июня. Нам говорят о контрреволюции в «Маленькой газете» и в «Новом времени». Но это не настоящая контрреволюция.  Это правильнее назвать [контрреволюционным хулиганством. Настоящая  контрреволюция гнездится в том классе, интересам которого вредит революция,  и этот класс есть буржуазия. Не случайность, что представителем буржуазии  является П.Н. Милюков. Сегодня буржуазия засвидетельствовала нам свою контрреволюционность. Из состава коалиционного министерства ушли министры-кадеты. Уход кадетов из министерства — это не доктринерство, как  хотят изобразить некоторые, а глубокое понимание своих классовых интересов.  [MASK] ведут страну к гибели. Об этом пишут в органе министров-социалистов,  в «Рабочей газете»...3 На точке зрения партии кадетов. Из-за чего, собственно, произошел последний министерский кризис? Кадетский министр, министр просвещения Мануйлов, отказался сместить чиновников контрреволюционеров в своем ведомстве. Впрочем, не сместили чиновников и министры-социалисты Керенский и Переверзев по министерству юстиции. Отказ Мануйлова подчистить свое министерство привел к тому, что кадеты ухватились, как за повод, за Украинскую раду и вышли из состава коалиционного министерства. А буржуазная печать разве не отравляет деревню?"
predict_masked_sent(text, top_k=5)

[MASK]: 'Они'  | weights: 0.6607780456542969
[MASK]: 'Мы'  | weights: 0.03076537884771824
[MASK]: 'Последние'  | weights: 0.02033715508878231
[MASK]: 'Коммунисты'  | weights: 0.01959843561053276
[MASK]: 'Все'  | weights: 0.01678912527859211


In [57]:
text = "Докладчиком по вопросу о борьбе с контрреволюцией выступает г. Зиновьев. - Он предлагает рабочей секции принять резолюцию, проект которой опубликован в газете «Правда» от 24 июня. Нам говорят о контрреволюции в «Маленькой газете» и в «Новом времени». Но это не настоящая контрреволюция.  Это правильнее назвать [контрреволюционным хулиганством. Настоящая  контрреволюция гнездится в том классе, интересам которого вредит революция,  и этот класс есть буржуазия. Не случайность, что представителем буржуазии  является П.Н. Милюков. Сегодня буржуазия засвидетельствовала нам свою контрреволюционность. Из состава коалиционного министерства ушли министры-кадеты. Уход кадетов из министерства — это не доктринерство, как  хотят изобразить некоторые, а глубокое понимание своих классовых интересов.  Капиталисты ведут страну к гибели. Об этом пишут в органе министров-социалистов,  в «Рабочей газете»...3 На точке зрения партии кадетов. Из-за чего, собственно, произошел последний министерский [MASK]? Кадетский министр, министр просвещения Мануйлов, отказался сместить чиновников контрреволюционеров в своем ведомстве. Впрочем, не сместили чиновников и министры-социалисты Керенский и Переверзев по министерству юстиции. Отказ Мануйлова подчистить свое министерство привел к тому, что кадеты ухватились, как за повод, за Украинскую раду и вышли из состава коалиционного министерства. А буржуазная печать разве не отравляет деревню?"
predict_masked_sent(text, top_k=5)

[MASK]: 'скандал'  | weights: 0.21997325122356415
[MASK]: 'переворот'  | weights: 0.1499113291501999
[MASK]: 'раскол'  | weights: 0.13498306274414062
[MASK]: 'мятеж'  | weights: 0.07210928946733475
[MASK]: 'расстрел'  | weights: 0.06264378130435944
